Compare NOAA Tides
================== 

This ({nb-download}`notebook <Compare-NOAA-Tides.ipynb>`) downloads and plots measurements and forecasts from [NOAA Tides and Currents](https://tidesandcurrents.noaa.gov/). Predictions use the harmonic constituents provided by the National Oceanic and Atmospheric Administration (NOAA) for the selected station.

## Python Dependencies

 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [pandas: Python Data Analysis Library](https://pandas.pydata.org/)

## Program Dependencies

- `arguments.py`: load the nodal corrections for tidal constituents  
- `astro.py`: computes the basic astronomical mean longitudes  
- `io.NOAA.py`: query and parsing functions for NOAA webservices API  
- `io.constituents.py`: basic tide model constituent class  
- `predict.py`: predict tidal values using harmonic constants  
- `time.py`: utilities for calculating time operations

```{note}
This notebook uses Jupyter widgets to set parameters for calculating the tidal time series.  
```

In [ ]:
import logging
import pyTMD.io
import timescale
import ipywidgets
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
# create logger
logging.basicConfig(level=logging.INFO)

## Query NOAA webservices for list of tide stations

In [ ]:
# get list of tide prediction stations
stations = pyTMD.io.NOAA.prediction_stations()

## Select NOAA station and dates for prediction

In [ ]:
# display widgets
TMDwidgets = pyTMD.tools.widgets()
# create dropdown with all tide prediction station data
TMDwidgets.stations = ipywidgets.Dropdown(
    options=stations.index,
    value='La Jolla (Scripps Institution Wharf)',
    description='Tide Stations:',
    disabled=False,
    style=TMDwidgets.style,
)
# create date pickers for start and end dates
end_date = pd.Timestamp.now().floor(freq='d')
start_date = end_date - pd.Timedelta('2 day')
TMDwidgets.start_date = ipywidgets.DatePicker(
    description='Start Date:',
    value=start_date,
    disabled=False,
    style=TMDwidgets.style,
)
TMDwidgets.end_date = ipywidgets.DatePicker(
    description='End Date:',
    value=end_date,
    disabled=False,
    style=TMDwidgets.style,
)
# display widgets
TMDwidgets.VBox([
    TMDwidgets.stations,
    TMDwidgets.start_date,
    TMDwidgets.end_date
])

## Predict tidal time series

In [ ]:
# get station name and indicator
station_row = stations.iloc[TMDwidgets.stations.index]
station_name = station_row.name
station_id = station_row.ID
# set query parameters
startdate = TMDwidgets.start_date.value.strftime('%Y%m%d')
enddate = TMDwidgets.end_date.value.strftime('%Y%m%d')

# get harmonic constituents for station
hcons = pyTMD.io.NOAA.harmonic_constituents(stationId=station_id)
# convert constituents to xarray Dataset
ds = hcons.tmd.to_dataset()

# get water levels for station and date range
api = 'waterlevelrawsixmin'
wlevel = pyTMD.io.NOAA.water_level(api, stationId=station_id,
    beginDate=startdate, endDate=enddate)

# predict tides at water level timestamps
ts = timescale.from_datetime(wlevel.timeStamp)
TIDE = ds.tmd.predict(ts.tide, deltat=ts.tt_ut1, corrections='GOT')
# infer minor constituents and add to major components
TIDE += ds.tmd.infer(ts.tide, deltat=ts.tt_ut1, corrections='GOT')

## Compare measured tide values with predictions

In [ ]:
fig, ax = plt.subplots(num=1)
ax.plot(wlevel.timeStamp, wlevel.WL, color='mediumseagreen', label='MSL')
ax.fill_between(wlevel.timeStamp, wlevel.WL-wlevel.sigma,
    y2=wlevel.WL+wlevel.sigma, zorder=1,
    color='mediumseagreen', alpha=0.35)
ax.plot(wlevel.timeStamp, TIDE, color='darkorchid', label='Tides')
ax.grid(linestyle='-', axis='x')
ax.set_title(f'{station_name} (id: {station_id})')
ax.set_ylabel('Water Level Height [m]')
lgd = ax.legend(frameon=True)
lgd.get_frame().set_boxstyle('square,pad=0.0')
lgd.get_frame().set_edgecolor("white")
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
date_formatter = mdates.DateFormatter("%Y-%m-%d %H:%M")
ax.xaxis.set_major_formatter(date_formatter)
fig.autofmt_xdate()
plt.show()